## Getting the required dataframe from table on the website.

In [2]:
import requests 
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
my_table = soup.find('table',{'class':'wikitable sortable'})
rows = my_table.findAll('tr')
table_content = ""
for table_rows in my_table.findAll("tr"):
    data = ""
    for table_data in my_table.findAll("td"):
        data = data + "," + table_data.text
    if len(data)!=0:
        table_content = table_content  + data[0:]

header = "Postalcode,Borough,Neighbourhood" + "\n"
file = open(os.path.expanduser("table.csv"),"wb")
file.write(bytes(header, encoding = "ascii", errors = "ignore"))
file.write(bytes(table_content, encoding = "ascii", errors = "ignore"))

toronto_df = pd.read_csv('table.csv')
toronto_df = toronto_df[toronto_df.Borough != 'NaN']
toronto_df.index = pd.RangeIndex(len(toronto_df.index))
toronto_df["Borough"] = toronto_df["Borough"].replace(["Not assigned"], np.nan)
toronto_df = toronto_df.dropna(subset = ["Borough"])
new_df=toronto_df.groupby("Postalcode").agg(lambda x:','.join(set(x)))
new_df.loc[new_df['Neighbourhood']=="Not assigned",'Neighbourhood']=new_df.loc[new_df['Neighbourhood']=="Not assigned",'Borough']
new_df.shape

(103, 2)

In [3]:
new_df.head()

,Borough,Neighbourhood
Postalcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


## Geospatial Data of Toronto

In [4]:
from geopy.geocoders import Nominatim 

In [6]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
new_df['Latitude']=geo_data['Latitude'].values
new_df['Longitude']=geo_data['Longitude'].values

new_df

,Borough,Neighbourhood,Latitude,Longitude
Postalcode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
